In [ ]:
!pip install flair

In [ ]:
import pandas as pd
dataset=pd.read_csv("tesr.csv")


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

# Assuming 'source' and 'target' are the columns containing text
source_text = dataset['source'].astype(str)  # Assuming df is your DataFrame
target_text = dataset['target'].astype(str)

# Tokenizing using NLTK
source_tokens = source_text.apply(word_tokenize)
target_tokens = target_text.apply(word_tokenize)

In [ ]:
import re

def remove_diacritics(tokens):
    clean_tokens = []
    for token in tokens:
        clean_token = re.sub(r'[\u064B-\u065F\u0610-\u061A]+', '', token)  # Remove Arabic diacritics
        clean_tokens.append(clean_token)
    return clean_tokens

source_tokens = source_tokens.apply(remove_diacritics)
target_tokens = target_tokens.apply(remove_diacritics)

In [ ]:
pip  install -U farasapy

In [ ]:

from farasa.stemmer import FarasaStemmer

# Initialize FarasaStemmer
farasa_stemmer = FarasaStemmer(interactive=True)

def apply_stemming(tokens):
    stemmed_tokens = []
    for token in tokens:
        stemmed_token = farasa_stemmer.stem(token)
        stemmed_tokens.append(stemmed_token)
    return stemmed_tokens

source_tokens = source_tokens.apply(apply_stemming)
target_tokens = target_tokens.apply(apply_stemming)

[2023-12-03 21:00:07,051 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [ ]:
from nltk.corpus import stopwords

nltk.download('stopwords')  # Download stopwords

# Arabic stopwords
arabic_stopwords = set(stopwords.words('arabic'))

def remove_stopwords(tokens):
    clean_tokens = [token for token in tokens if token not in arabic_stopwords]
    return clean_tokens

source_tokens = source_tokens.apply(remove_stopwords)
target_tokens = target_tokens.apply(remove_stopwords)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def remove_blanks(tokens):
    return [token for token in tokens if token.strip()]

source_tokens = source_tokens.apply(remove_blanks)
target_tokens = target_tokens.apply(remove_blanks)

In [ ]:
# Printing preprocessed tokens from 'source' and 'target' columns
for idx in range(len(dataset)):  # Assuming 'df' is your DataFrame
    print(f"Source Text {idx + 1}:")
    print(source_tokens[idx])
    print("\nTarget Text:")
    print(target_tokens[idx])
    print("\n---------------------\n")

Source Text 1:
['بسم', 'الله', 'رحمن', 'رحيم']

Target Text:
['جاء', 'رسول', 'عزيز', 'عنت', 'حريص', 'مؤمن', 'رءوف', 'رحيم']

---------------------

Source Text 2:
['بسم', 'الله', 'رحمن', 'رحيم']

Target Text:
['قال', 'اركب', 'بسم', 'الله', 'مجرى', 'مرسا', 'رب', 'غفور', 'رحيم']

---------------------

Source Text 3:
['بسم', 'الله', 'رحمن', 'رحيم']

Target Text:
['نعمة', 'الله', 'مس', 'ضر', 'جأر']

---------------------

Source Text 4:
['بسم', 'الله', 'رحمن', 'رحيم']

Target Text:
['رحمن', 'عرش', 'استوى']

---------------------

Source Text 5:
['بسم', 'الله', 'رحمن', 'رحيم']

Target Text:
['سبح', 'اسم', 'رب', 'عظيم']

---------------------

Source Text 6:
['حمد', 'الله', 'رب', 'عالم']

Target Text:
['قال', 'فرعون', 'رب', 'عالم']

---------------------

Source Text 7:
['مالك', 'يوم', 'دين']

Target Text:
['ورث', 'أرض']

---------------------

Source Text 8:
['مالك', 'يوم', 'دين']

Target Text:
['ملك', 'يومئذ', 'حق', 'رحمن', 'يوم', 'كافر', 'عسير']

---------------------

Source Text 9:
['م

code


In [ ]:
# Assuming source_tokens, target_tokens, and relevance_degrees are available
relevance_degree=dataset["relevance_degree"]
# Combine preprocessed texts and relevance degrees
combined_data = []
for idx in range(len(source_tokens)):
    combined_data.append({
        'source': ' '.join(source_tokens[idx]),
        'target': ' '.join(target_tokens[idx]),
        'relevance_degree': relevance_degree[idx]
    })


In [ ]:

from flair.data import Sentence
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import spearmanr
from flair.embeddings import DocumentPoolEmbeddings, WordEmbeddings, FlairEmbeddings






# Initialize Flair embeddings
word_embedding = WordEmbeddings('ar')
flair_embedding_forward = FlairEmbeddings('ar-forward')
flair_embedding_backward = FlairEmbeddings('ar-backward')

document_embeddings = DocumentPoolEmbeddings([
    word_embedding, flair_embedding_forward, flair_embedding_backward
])



# Calculate cosine similarity between source and target for each row
for idx, data in enumerate(combined_data):
    source_text = data['source']
    target_text = data['target']

    # Create Sentence objects for source and target texts
    source_sent = Sentence(source_text)
    target_sent = Sentence(target_text)

    # Embed source and target texts
    document_embeddings.embed(source_sent)
    document_embeddings.embed(target_sent)

    # Extract embeddings
    source_embedding = source_sent.embedding.detach().cpu().numpy().reshape(1, -1)
    target_embedding = target_sent.embedding.detach().cpu().numpy().reshape(1, -1)

    print(f"Row {idx + 1}:")
    print(f"Source Embedding: {source_embedding}")
    print(f"Target Embedding: {target_embedding}")
    print("\n")


Row 1:
Source Embedding: [[-0.07250576 -0.04915975  0.17958775 ... -0.00729184 -0.00264265
   0.02988276]]
Target Embedding: [[-0.00026038 -0.08274475  0.123991   ... -0.00875888 -0.01908972
   0.03479582]]


Row 2:
Source Embedding: [[-0.07250576 -0.04915975  0.17958775 ... -0.00729184 -0.00264265
   0.02988276]]
Target Embedding: [[-0.02435     0.14020087  0.15966211 ...  0.00051894 -0.00463251
   0.02345513]]


Row 3:
Source Embedding: [[-0.07250576 -0.04915975  0.17958775 ... -0.00729184 -0.00264265
   0.02988276]]
Target Embedding: [[ 0.0468092  -0.0371162  -0.07604159 ...  0.00951916  0.00847298
   0.01452282]]


Row 4:
Source Embedding: [[-0.07250576 -0.04915975  0.17958775 ... -0.00729184 -0.00264265
   0.02988276]]
Target Embedding: [[-0.13898666  0.04133    -0.00679634 ... -0.00414237  0.00942603
   0.01506603]]


Row 5:
Source Embedding: [[-0.07250576 -0.04915975  0.17958775 ... -0.00729184 -0.00264265
   0.02988276]]
Target Embedding: [[ 0.08079375  0.21331926 -0.23806101 .

In [ ]:

from flair.data import Sentence
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import spearmanr
from flair.embeddings import DocumentPoolEmbeddings, WordEmbeddings, FlairEmbeddings






# Initialize Flair embeddings
word_embedding = WordEmbeddings('ar')
flair_embedding_forward = FlairEmbeddings('ar-forward')
flair_embedding_backward = FlairEmbeddings('ar-backward')

document_embeddings = DocumentPoolEmbeddings([
    word_embedding, flair_embedding_forward, flair_embedding_backward
])



# Calculate cosine similarity between source and target for each row
for idx, data in enumerate(combined_data):
    source_text = data['source']
    target_text = data['target']

    # Create Sentence objects for source and target texts
    source_sent = Sentence(source_text)
    target_sent = Sentence(target_text)

    # Embed source and target texts
    document_embeddings.embed(source_sent)
    document_embeddings.embed(target_sent)

    # Extract embeddings
    source_embedding = source_sent.embedding.detach().cpu().numpy().reshape(1, -1)
    target_embedding = target_sent.embedding.detach().cpu().numpy().reshape(1, -1)

    # Calculate cosine similarity between source and target embeddings
    cos_sim = cosine_similarity(source_embedding, target_embedding)[0][0]

    print(f"Row {idx+1}: Cosine Similarity between Source and Target: {cos_sim}")


Row 1: Cosine Similarity between Source and Target: 0.6397463083267212
Row 2: Cosine Similarity between Source and Target: 0.7611216902732849
Row 3: Cosine Similarity between Source and Target: 0.5671250820159912
Row 4: Cosine Similarity between Source and Target: 0.5865022540092468
Row 5: Cosine Similarity between Source and Target: 0.5186315178871155
Row 6: Cosine Similarity between Source and Target: 0.7383261919021606
Row 7: Cosine Similarity between Source and Target: 0.38394513726234436
Row 8: Cosine Similarity between Source and Target: 0.6362152695655823
Row 9: Cosine Similarity between Source and Target: 0.493289053440094
Row 10: Cosine Similarity between Source and Target: 0.6455408334732056
Row 11: Cosine Similarity between Source and Target: 0.7351135015487671
Row 12: Cosine Similarity between Source and Target: 0.5874470472335815
Row 13: Cosine Similarity between Source and Target: 0.38847678899765015
Row 14: Cosine Similarity between Source and Target: 0.3504643142223358


In [ ]:

from flair.data import Sentence
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import spearmanr
from flair.embeddings import DocumentPoolEmbeddings, WordEmbeddings, FlairEmbeddings




# Initialize lists to store cosine similarities and relevance degrees
cos_similarities = []
relevance_degrees = []

# Initialize Flair embeddings
word_embedding = WordEmbeddings('ar')
flair_embedding_forward = FlairEmbeddings('ar-forward')
flair_embedding_backward = FlairEmbeddings('ar-backward')

document_embeddings = DocumentPoolEmbeddings([
    word_embedding, flair_embedding_forward, flair_embedding_backward
])

# Calculate cosine similarity between source and target for each row
for idx, data in enumerate(combined_data):
    source_text = data['source']
    target_text = data['target']

    # Create Sentence objects for source and target texts
    source_sent = Sentence(source_text)
    target_sent = Sentence(target_text)

    # Embed source and target texts
    document_embeddings.embed(source_sent)
    document_embeddings.embed(target_sent)

    # Extract embeddings
    source_embedding = source_sent.embedding.detach().cpu().numpy().reshape(1, -1)
    target_embedding = target_sent.embedding.detach().cpu().numpy().reshape(1, -1)

    # Calculate cosine similarity between source and target embeddings
    cos_sim = cosine_similarity(source_embedding, target_embedding)[0][0]

     # Append cosine similarity to the list
    cos_similarities.append(cos_sim)

    # Append relevance degree to the list
    relevance_degrees.append(data['relevance_degree'])

# Calculate Spearman correlation between cosine similarities and relevance degrees
spearman_corr, spearman_p_value = spearmanr(cos_similarities, relevance_degrees)

print(f"Spearman Correlation: {spearman_corr} (p-value: {spearman_p_value})")



2023-12-03 20:25:21,547 https://flair.informatik.hu-berlin.de/resources/embeddings/token/ar-wiki-fasttext-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmprfja078x


100%|██████████| 699M/699M [00:55<00:00, 13.2MB/s]

2023-12-03 20:26:17,934 copying /tmp/tmprfja078x to cache at /root/.flair/embeddings/ar-wiki-fasttext-300d-1M.vectors.npy


2023-12-03 20:26:20,381 removing temp file /tmp/tmprfja078x
2023-12-03 20:26:21,568 https://flair.informatik.hu-berlin.de/resources/embeddings/token/ar-wiki-fasttext-300d-1M not found in cache, downloading to /tmp/tmpaxugphsc


100%|██████████| 25.5M/25.5M [00:03<00:00, 8.04MB/s]

...

2023-12-03 20:26:46,319 copying /tmp/tmppg5ixjna to cache at /root/.flair/embeddings/lm-ar-opus-large-forward-v0.1.pt


2023-12-03 20:26:46,502 removing temp file /tmp/tmppg5ixjna
2023-12-03 20:26:47,799 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-ar-opus-large-backward-v0.1.pt not found in cache, downloading to /tmp/tmpd5_7typ_


100%|██████████| 126M/126M [00:10<00:00, 12.1MB/s]

2023-12-03 20:26:59,487 copying /tmp/tmpd5_7typ_ to cache at /root/.flair/embeddings/lm-ar-opus-large-backward-v0.1.pt


2023-12-03 20:26:59,646 removing temp file /tmp/tmpd5_7typ_
Spearman Correlation: 0.5533491168726251 (p-value: 1.9437660441685625e-12)


In [ ]:

from flair.data import Sentence
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr
from flair.embeddings import DocumentPoolEmbeddings, WordEmbeddings, FlairEmbeddings

# Initialize Flair embeddings
word_embedding = WordEmbeddings('ar')
flair_embedding_forward = FlairEmbeddings('ar-forward')
flair_embedding_backward = FlairEmbeddings('ar-backward')

document_embeddings = DocumentPoolEmbeddings([
    word_embedding, flair_embedding_forward, flair_embedding_backward
])

# Calculate cosine similarity between source and target for each row
for idx, data in enumerate(combined_data):
    source_text = data['source']
    target_text = data['target']

    # Create Sentence objects for source and target texts
    source_sent = Sentence(source_text)
    target_sent = Sentence(target_text)

    # Embed source and target texts
    document_embeddings.embed(source_sent)
    document_embeddings.embed(target_sent)

    # Extract embeddings
    source_embedding = source_sent.embedding.detach().cpu().numpy().reshape(1, -1)
    target_embedding = target_sent.embedding.detach().cpu().numpy().reshape(1, -1)

    # Calculate cosine similarity between source and target embeddings
    cos_sim = cosine_similarity(source_embedding, target_embedding)[0][0]

    # Append cosine similarity to the list
    cos_similarities.append(cos_sim)

    # Append relevance degree to the list
    relevance_degrees.append(data['relevance_degree'])

# Calculate Pearson correlation between cosine similarities and relevance degrees
pearson_corr, pearson_p_value = pearsonr(cos_similarities, relevance_degrees)

print(f"Pearson Correlation: {pearson_corr} (p-value: {pearson_p_value})")


Pearson Correlation: 0.5459341535825786 (p-value: 4.356888339754758e-12)
